#### Importing relevant packages

In [4]:
import pandas as pd
from pynancial_steering import Redshift
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta

#### Redshift connection

In [ ]:
# connect to Redshift
lh = Redshift()

####  Getting input dates for observation window
####  Then we calculate the max window we need to have i.e. observation end date +7

In [5]:
#format dd/mm/yy
start_date = '2022-01-02'
# print(start_date)
end_date = '2022-01-09'
# print(end_date)

date1 = datetime.strptime(start_date, "%Y-%m-%d")
date2 = datetime.strptime(end_date,"%Y-%m-%d")
date3 = date2 + timedelta(days=8)

window_start_date = date1.strftime("%Y-%m-%d")
window_end_date = date3.strftime("%Y-%m-%d")

print(window_start_date)
print(window_end_date)
print(date1)
print(date3)

2022-01-02
2022-01-17
2022-01-02 00:00:00
2022-01-17 00:00:00


Creating a list of dates from the observation window \
We will use this list to iterate over the observation window data later on

In [6]:
date_list = [date1 + timedelta(days=x) for x in range (((date2 + timedelta(days=1))-date1).days)]
date_list

[datetime.datetime(2022, 1, 2, 0, 0),
 datetime.datetime(2022, 1, 3, 0, 0),
 datetime.datetime(2022, 1, 4, 0, 0),
 datetime.datetime(2022, 1, 5, 0, 0),
 datetime.datetime(2022, 1, 6, 0, 0),
 datetime.datetime(2022, 1, 7, 0, 0),
 datetime.datetime(2022, 1, 8, 0, 0),
 datetime.datetime(2022, 1, 9, 0, 0)]

Pulling data from lakehouse 

In [ ]:
df_lh=lh.run_query(
    f"COMMIT;"
    
    f"select occurred_at, user_attributes_market, customer_id, event_name from "
    f"xx_yy.zz_kk "
    f"where user_attributes_market='US' and event_name in "
    f"('Help me', 'Entertain you')"
    f" and occurred_at between '{date1}' and '{date3}' "
    f"limit 100000" 
    );

### Filtering and creating 2 separate data frames for each event and substring to create date from timestamp

In [ ]:
# filter first
df_vcn = df_lh.loc[df_lh['event_name'] == 'Help me']
df_vcn['amp_date'] = df_vcn['occurred_at'].astype(str).str.slice(0,10)


df_tdp = df_lh.loc[df_lh['event_name'] == 'Entertain you']
df_tdp['amp_date'] = df_tdp['occurred_at'].astype(str).str.slice(0,10)

1. Now we iterate over the list of observation window dates one at a time  
2. We first pass this date to our VCN dataframe and pull out all records for one date at a time 
3. Then we use this window date +7 to full all relevant records from the tDP dataframe  
4. Nxt we sum up the number of times the Open TDP event happened at a user level  
5. Left merge/join to the dataframe in step 2 
6. Append/concatenate to empty (first iteration)/union file

In [ ]:
for i in range(len(date_list)):
    df_vcn_1 = df_vcn[df_vcn['amp_date'] == date_list[i].strftime("%Y-%m-%d").drop("occurred_at",1)
                    
    df_vcn_2 = df_vcn_1.drop_duplicates(["customer_id","event_name","amp_date"],keep="first")
                      
    
                      
    df_tdp_1 = df_tdp[df_tdp['amp_date'] >= date_list[i].strftime("%Y-%m-%d")
                     & (df_tdp['amp_date'] <= (date_list[i] + timedelta(days=7)).strftime("%Y-%m-%d"))]
                      
    df_tdp_2 = df_tdp_1.groupby(["customer_id"],sort=True)['event_name'].count().reset_index(name="count_tdp_view")
    
    merge_1 = pd.merge(df_vcn_2.astype(str),df_tdp_2.astype(str), how = 'left', on = 'customer_id')
                      
    if i == 0:
            df_union = merge_1
    
    else:
        df_union = df_union.append(merge_1, ignore_index = True)

In [ ]:
df_union.describe()

In [ ]:
df_union.head()

In [ ]:
df_union['count_tdp_view'].unique()

#### 1. Fillna to substitute Nan values with zeroes
#### 2. Groupby to count number of users by number of times that the TDP event happened in the nxt 7 days
#### 3. Correcting messed up formats
#### 4. Sorting in ascending order 

In [ ]:
# Final table

df_union['count_tdp_view'] = df_union['count_tdp_view'].fillna()
df_final_1 = df_union.groupby("count_tdp_view", sort=True)['customer_id'].count().reset_index(name="count_users")
df_final_1['count_tdp_view'] = df_final_1['count_tdp_view'].astype('int')
df_final_1.sort_values(by=['count_tdp_view'])